In [ ]:
# We start by getting access to the drive
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
import sys,os
# change directory
%cd '/content/drive/My Drive'
# Let's import the python files to have access to its functions and Classes
path_to_module='/content/drive/My Drive/segmentation_codes/'
sys.path.append(os.path.abspath(path_to_module))
import seg_utils, seg_config_bin, seg_model

/content/drive/My Drive


In [ ]:
import torch
import cv2
import os
from PIL import Image
from google.colab.patches import cv2_imshow

In [ ]:
out_dir = '/content/drive/My Drive/segmentation_codes/outputs_bin/'
out_dir = os.path.join(out_dir, 'inference_result_Google')
os.makedirs(out_dir, exist_ok=True)

# Set computation device.jpg
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = seg_model.prepare_model(len(seg_config_bin.ALL_CLASSES))
ckpt = torch.load('/content/drive/My Drive/segmentation_codes/outputs_bin/best_model_seg.pth', map_location=device)
model.load_state_dict(ckpt['model_state_dict'])
model.eval().to(device)

input = '/content/drive/My Drive/segmentation_codes/test_Google/'
all_image_paths = os.listdir(input)
for i, image_path in enumerate(all_image_paths):
    print(f"Image {i+1}")
    # Read the image.
    image = Image.open(os.path.join(input, image_path))

    # Resize very large images (if width > 1024.) to avoid OOM on GPUs.
    if image.size[0] > 1024:
        image = image.resize((800, 800))

    # Do forward pass and get the output dictionary.
    outputs = seg_utils.get_segment_labels(image, model, device)
    # Get the data from the `out` key.
    outputs = outputs['out']
    segmented_image = seg_utils.draw_segmentation_map(outputs)

    final_image = seg_utils.image_overlay(image, segmented_image)
    # cv2_imshow(final_image)
    image_path = image_path[:-4] + '.png'
    cv2.waitKey(1)
    cv2.imwrite(os.path.join(out_dir, image_path), final_image)

Downloading: "https://download.pytorch.org/models/deeplabv3_resnet50_coco-cd0a2569.pth" to /root/.cache/torch/hub/checkpoints/deeplabv3_resnet50_coco-cd0a2569.pth
100%|██████████| 161M/161M [00:00<00:00, 216MB/s]


Image 1
